In [1]:
from serlo import SerloBot, Environment

bot = SerloBot(Environment.STAGING)
bot.login()

Serlo-Username: ········
Serlo-Password: ········


In [4]:
import json
import mysql.connector

from serlo.mysql import query
from IPython.display import display, Markdown, HTML

def getAllEntityRevisions():
    results = query("""
        select
            max(entity_revision.id),
            max(entity.id),
            max(type.name),
            json_objectagg(entity_revision_field.field, entity_revision_field.value) as fields
        from entity_revision
            join entity on entity_revision.repository_id = entity.id
            join type on entity.type_id = type.id
            join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        group by entity_revision.id
    """)
    
    return [{
        "revisionId": result[0],
        "entityId": result[1],
        "repositoryId": result[1],
        "type": result[2],
        **json.loads(result[3])
    } for result in results]

getAllEntityRevisions()[0]

{'revisionId': 1496,
 'entityId': 1495,
 'repositoryId': 1495,
 'type': 'article',
 'title': 'Addition',
 'content': '[[{"col":24,"content":"Die **Addition**, umgangssprachlich auch **Plus-Rechnen** genannt, ist eine der vier  [Grundrechenarten](/1871) . In der Grundschule und in der Umgangssprache verwendet man meist den Ausdruck **Zusammenz\\u00e4hlen** f\\u00fcr die Addition von zwei oder mehr Zahlen, da Addition den Vorgang des Z\\u00e4hlens beschreibt.\\n\\n\\u00a0\\n\\nDie Elemente bzw. Operanden einer Addition werden\\u00a0**Summanden**\\u00a0und das Ergebnis\\u00a0**Summe**\\u00a0genannt:\\u00a0\\n\\n1. Summand + 2. Summand = Summe\\n\\nEine Summe muss aber nicht nur aus 2 Summanden bestehen, sie kann auch aus mehreren Summanden bestehen.\\u00a0\\n\\n\\u00a0\\n\\nIm Allgemeinen ist die Addition nicht f\\u00fcr f\\u00fcr Zahlen definiert. Vektoren kann man zum Beispiel auch miteinander addieren. Die Grundrechenart, die eine Addition \\"r\\u00fcckg\\u00e4ngig\\" macht, ist\\u00a0

In [5]:
def getAllPageRevisions():
    results = query("""
        select
            id, page_repository_id, title, content, date, trashed
        from page_revision
    """)
    
    return [{
        "revisionId": result[0],
        "repositoryId": result[1],
        "title": result[2],
        "content": result[3],
        "date": result[4],
        "trashed": result[5],
        "type": "page"
    } for result in results]

getAllPageRevisions()[0]

{'revisionId': 16258,
 'repositoryId': 16256,
 'title': 'Das Team',
 'content': '[[{"col":24,"content":"Einen maÃ\x9fgeblichen Beitrag zu Serlo leisten die aktiven Nutzer, die sich mit der Ausarbeitung und Verbesserung  von LÃ¶sungen und Artikeln beschÃ¤ftigen. Jeder, der aktiv an Serlo arbeitet, kann Mitglied in dem Verein werden. Serlo ist Gemeingut und gehÃ¶rt keinem festen Personenkreis.\\n\\nOrganisation\\n"}],[{"col":6,"content":"![Simon KÃ¶hl](/attachment/file/16257/5)\\n>[zum Profil](10)"},{"col":6,"content":""},{"col":6,"content":""},{"col":6,"content":""}]]',
 'date': datetime.datetime(2014, 3, 7, 0, 28, 9),
 'trashed': 0,
 'type': 'page'}

In [7]:
def parseEdtrState(content):
    try:
        state = json.loads(content)
    except:
        return None
    
    if "plugin" not in state:
        return None
    
    return state

def stateHasPlugin(plugin, state):
    if isinstance(state, dict):
        if "plugin" in state and state["plugin"] == plugin:
            return True
        
        return any(stateHasPlugin(plugin, value) for value in state.values())
    elif isinstance(state, list):
        return any(stateHasPlugin(plugin, value) for value in state)
    else:
        return False

def revisionHasPlugin(plugin, revision):
    if "content" not in revision:
        return False
    
    state = parseEdtrState(revision["content"])
    
    return stateHasPlugin(plugin, state) if state != None else False

In [9]:
revisions = getAllPageRevisions() + getAllEntityRevisions()
affected_revisions = (r for r in revisions if revisionHasPlugin("important", r) or revisionHasPlugin("blockquote", r))
affected_ids = [106081, 41107] + [r["revisionId"] for r in affected_revisions]

len(affected_ids)

2983

In [11]:
def uuid_key(uuid):
    return "de.serlo.org/api/uuid/%s" % uuid

def update_cache(ids, chunk_size=100):
    chunks = [ids[i:i+chunk_size] for i in range(0, len(ids), chunk_size)]
    
    for chunk in chunks:
        result = bot.api_call("""
            mutation($input: CacheUpdateInput!) {
                _cache {
                    update(input: $input) {
                        success
                    }
                }
            }
        """, variables= {
            "input": {
                "keys": [uuid_key(uuid) for uuid in chunk]
            }
        })
        
        print(result)

update_cache(affected_ids)

{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': True}}}
{'_cache': {'update': {'success': 